In [1]:
import matplotlib.pyplot as plt, numpy as np, noise
from skimage import io
from skimage.morphology import (disk, ball)
from skimage.filters import roberts

In [2]:
# Set Radius of island and island seed, plus misc options
seed = 14
r = 100
pond_cutoff = 10
rook_adjacency = False
truncation = False

In [3]:
# Initializing some things for later
d = (r*2)+1
ponds = []
pond_pixels = []

In [4]:
# Define Pixel class for each tile of heightmap
class Pixel:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        self.vol = 0.1
        self.pointer = self
    
    # Convenience method for printing
    def coordinates(self):
        return (self.x, self.y, self.z)
    
    # Method which returns list of pixels adjacent to a pixel, including the pixel itself
    def get_adjacents(self):
        adjacents = []
        if rook_adjacency == False:
            for i in range(3):
                for j in range(3):
                    adjacents.append(pixel_array[self.y-1+i][self.x-1+j])
        # alternate get_adjacents which doesn't count diagonals, triggered by turning setting rook_adjacency = True
        else:
            adjacents.append(pixel_array[self.y-1][self.x])
            adjacents.append(pixel_array[self.y+1][self.x])
            adjacents.append(pixel_array[self.y][self.x-1])
            adjacents.append(pixel_array[self.y][self.x+1])
            adjacents.append(self)
        return adjacents
    
    # Method which determines which pixel the pixel will outflow to
    def update_pointer(self):
        to_beat = 999999999
        lowest = self
        for adj in self.get_adjacents():
            if adj.z < to_beat:
                to_beat = adj.z
                lowest = adj
        self.pointer = lowest

In [5]:
# Pond Class definition
class Pond:
    def __init__(self, origin):
        self.members = [origin]
        self.z = origin.z
        self.pointer = origin
        self.vol = 0
        self.edges = [origin]
        self.update_edges()
    
    # Method which determines which pixels are adjacent to the pond, but not members of it
    def update_edges(self):
        adjs = self.pointer.get_adjacents()
        for adj in adjs:
            if (adj not in self.edges) and (adj not in self.members) and (adj in pixel_list):
                self.edges.append(adj)
        self.edges.remove(self.pointer)
    
    # Method which determines which pixel the pond will outflow to (the lowest edge)
    def update_pond_pointer(self):
        to_beat = 999999999
        lowest = self.members[0]
        for edge in self.edges:
            if edge.z <= to_beat:
                to_beat = edge.z
                lowest = edge
        self.pointer = lowest
    
    # Method which raises a pond's level until it finds and outflow point (i.e. an adjacent pixel lower than the pond level)
    def rise(self):
        last_pond = self
        while True:
            self.update_pond_pointer()
            if self.pointer.z >= self.z:
                
                # Pond collision loop
                for pond in ponds:
                    if pond == self:
                        continue
                    if (self.pointer in pond.members):
                        pond.members += self.members
                        for edge in self.edges:
                            if (edge not in pond.edges) and (edge not in pond.members):
                                pond.edges.append(edge)
                        to_remove = []
                        for edge in pond.edges:
                            if (edge in self.members):
                                to_remove.append(edge)
                        for edge in to_remove:
                            pond.edges.remove(edge)
                        ponds.remove(self)
                        last_pond = pond.rise()
                        break
                
                # Rise update if pond doesn't collide
                if self in ponds:
                    self.update_edges()
                    self.z = self.pointer.z
                    self.members.append(self.pointer)
                    if self.pointer in self.edges:
                        self.edges.remove(self.pointer)
                else:
                    break
            else:
                break
        return last_pond
        
    # Method to determine the volume - NOT level - of a pond. Volume represents the total flowthrough, not pond depth
    def vol_update(self):
        self.vol = 0.1*len(self.members)
        inflows = self.edges
        for inflow in inflows:
            if (inflow.pointer in self.members) and (inflow not in pond_pixels):
                self.vol += inflow.vol

In [6]:
# This cell adapted from https://medium.com/@yvanscher/playing-with-perlin-noise-generating-realistic-archipelagos-b59f004d8401 to generate perlin using Noise module
shape = (d,d)
scale = r
octaves = 6
persistence = 0.5
lacunarity = 2.0

world = np.zeros(shape)
for i in range(d):
    for j in range(d):
        world[i][j] = noise.pnoise2(i/scale, 
                                    j/scale, 
                                    octaves=octaves, 
                                    persistence=persistence, 
                                    lacunarity=lacunarity, 
                                    repeatx=1024, 
                                    repeaty=1024, 
                                    base=seed)
        
plt.imshow(world)
plt.colorbar();

In [7]:
# Generate ball, flatten to 2d array
baseball = ball(r)
flatball = np.sum(baseball, axis=1)/(d-1)

In [8]:
# # Generate ball, flatten to 2d array - alternate version to limit roundness, less successful
# alternate_baseball = ball(r*2)#[:r+1]
# alternate_baseball = np.sum(alternate_baseball, axis=0)/(r-1)
# flatball = np.zeros(shape)
# for i in range(d):
#     for j in range(d):
#         flatball[i][j] = alternate_baseball[r+i][r+j]

In [9]:
# # Experimental cell to decrease "cliff" at edges of ball
# for i in range(d):
#     for j in range(d):
#         if flatball[i][j] <= 1/(d-1):
#             flatball[i][j] = 29/(d-1)

In [10]:
# Truncation, optional, triggered by setting truncation=True at the top
if truncation:
    for i in range(d):
        for j in range(d):
            if flatball[i][j] > 0.6:
                flatball[i][j] = 0.6

In [11]:
# Combine world and dome to create textured dome - code to display result commented out
noiseball = world+flatball
plt.imshow(noiseball)
plt.colorbar();

In [12]:
## Test cell - commented out in ordinary functioning

## Load test image
# test_island = io.imread('./TestIsland3.png')

## Convert image to 2d heightmap array
# d = len(test_island)
# shape = (d,d)
# test_world = np.zeros(shape)
# for i in range(d):
#     for j in range(d):
#         test_world[i][j] = test_island[i][j][0]

## Adjustments made to array as needed for specific tests
# test_world[2][4] = 96
# test_world[4][6] = 129

## Display test array
# plt.imshow(test_world)
# plt.colorbar();

## Overwrite island base with test array
# noiseball = test_world

In [13]:
# Set inundation level based on median altitude and put elevations in terms of sea level
flood = np.median(noiseball)
for i in range(d):
    for j in range(d):
        noiseball[i][j] -= flood

In [14]:
# Inundation visualization - purely graphical
floodball = np.zeros((d,d))
for i in range(d):
    for j in range(d):
        if noiseball[i][j] < 0:
            floodball[i][j] = -1
        else:
            floodball[i][j] = noiseball[i][j]

plt.imshow(floodball)
plt.colorbar();

In [15]:
# Create array of pixel objects from heightmap
pixel_array = np.empty(noiseball.shape, Pixel)
for i in range(d):
    for j in range(d):
        pixel_array[i][j] = (Pixel(j, i, noiseball[i][j]))

In [16]:
# Create list of pixels from array, excluding array border pixels
pixel_list = []
for i in range(1,(d-1)):
    for j in range(1,(d-1)):
        pixel_list.append(pixel_array[i][j])

In [17]:
# Sort list of pixels by elevation
pixel_list.sort(key=lambda a : a.z, reverse=True)

In [18]:
# Initial Flow - update pointers and add volume of each pixel to its pointer
for pixel in pixel_list:
    pixel.update_pointer()
    if pixel.pointer != pixel:
        pixel.pointer.vol += pixel.vol

In [19]:
# Initial Pond Creation
for pixel in reversed(pixel_list):
    if (pixel.pointer == pixel) and (pixel.vol >= pond_cutoff) and (pixel.z >= 0):
        new_pond = Pond(pixel)
        ponds.append(new_pond)
        new_pond.rise()
        new_pond.vol_update()

In [20]:
# Full Visualization
viz_array = np.zeros((d,d))
for i in range(d):
    for j in range(d):
        if pixel_array[i][j].vol > 100:
            viz_array[i][j] = 100
        else:
            viz_array[i][j] = pixel_array[i][j].vol
for pond in ponds:
    for member in pond.members:
        if pond.vol > 100:
            viz_array[member.y][member.x] = 100
        else:
            viz_array[member.y][member.x] = pond.vol
for i in range(d):
    for j in range(d):
        if pixel_array[i][j].z <= 0:
            viz_array[i][j] = 0

# back-up array for use in update visualization if needed
viz_array_1 = viz_array

plt.figure(figsize=(10,10))
plt.imshow(viz_array)
plt.colorbar();

In [21]:
# Creation of outflow array to determine locations of new ponds formed from pond outflows
outflow_array = np.zeros(shape)
outflow_array_1 = np.copy(outflow_array)

# Outflow from initial ponds
for pond in ponds:
    step = pond.pointer
    step2 = step.pointer
    outflow_array[step.y][step.x] += pond.vol
    while (step != step2) and (step2 in pixel_list):
        outflow_array[step2.y][step2.x] += outflow_array[step.y][step.x]
        step = step2
        step2 = step.pointer
    for member in pond.members:
        if member not in pond_pixels:
            pond_pixels.append(member)

# Create new ponds from outflow, run outflow from those ponds, and repeat until array stabilizes
while np.sum(outflow_array != outflow_array_1) != 0:
    outflow_array_1 = np.copy(outflow_array)
    for pixel in reversed(pixel_list):
        if (pixel.pointer == pixel) and (outflow_array[pixel.y][pixel.x] != 0) and (pixel not in pond_pixels) and (pixel.z >= 0):
            new_pond = Pond(pixel)
            ponds.append(new_pond)
            new_pond = new_pond.rise()
            new_pond.vol_update()
            step = new_pond.pointer
            step2 = step.pointer
            outflow_array[step.y][step.x] += new_pond.vol
            while (step != step2) and (step2 in pixel_list):
                outflow_array[step2.y][step2.x] += outflow_array[step.y][step.x]
                step = step2
                step2 = step.pointer
            
            for member in new_pond.members:
                if member not in pond_pixels:
                    pond_pixels.append(member)

In [22]:
# Visualization of outflows
for i in range(d):
    for j in range(d):
        if outflow_array[i][j] > 10:
            outflow_array[i][j] = 10
        
plt.figure(figsize=(10,10))        
plt.imshow(outflow_array)
plt.colorbar();

In [23]:
# Combination of outflows, ponds, and initial flows
viz_array = np.zeros((d,d))
for i in range(d):
    for j in range(d):
        if pixel_array[i][j].vol > 1000:
            viz_array[i][j] = 1000
        else:
            viz_array[i][j] = pixel_array[i][j].vol
for pond in ponds:
    for member in pond.members:
        viz_array[member.y][member.x] = pond.vol

viz_array_combined = outflow_array+viz_array
for i in range(d):
    for j in range(d):
        if pixel_array[i][j].z <= 0:
            viz_array_combined[i][j] = 0
        if viz_array_combined[i][j] > 10:
            viz_array_combined[i][j] = 10

plt.figure(figsize=(10,10))
plt.imshow(viz_array_combined)
plt.colorbar();

In [24]:
# Re-Flow array created so that volumes can be recalculated with all ponds in place
reflow_array = np.zeros(shape)

# Rainfall - volume reset and initial flow
for pixel in pixel_list:
    pixel.vol = 0.1
    pixel.update_pointer()
for pixel in pixel_list:
    if pixel.pointer != pixel:
        pixel.pointer.vol += pixel.vol

# Initial vol set for all ponds from rainfall
for pond in ponds:
    pond.vol_update()

# Ponds cascade from top to bottom BY LEVEL and store cascade in blank array    
for pond in sorted(ponds, key=lambda a : a.z, reverse=True):
    for pond_edge in pond.edges:
        if (pond_edge.pointer in pond.members) and (pond_edge not in pond_pixels):
            pond.vol += reflow_array[pond_edge.y][pond_edge.x]
    step = pond.pointer
    step2 = step.pointer
    if step in pond_pixels:
        for pondy in ponds:
            if step in pondy.members:
                pondy.vol += pond.vol
                break
        continue
    reflow_array[step.y][step.x] += pond.vol
    while (step != step2) and (step2 in pixel_list):
        reflow_array[step2.y][step2.x] += reflow_array[step.y][step.x]
        step = step2
        step2 = step.pointer
        
# Rainfall vols added to cascade array
for pixel in pixel_list: 
    reflow_array[pixel.y][pixel.x] += pixel.vol
    
# Set pond pixel vols to pond vol in cascade array
for pond in ponds:
    for member in pond.members:
        reflow_array[member.y][member.x] = pond.vol

In [25]:
# Visualization of final volumes
better_viz_array = np.copy(reflow_array)

# Set ocean pixels to constant and cap vols if desired
for i in range(d):
    for j in range(d):
        if pixel_array[i][j].z <= 0:
            better_viz_array[i][j] = 0
        if better_viz_array[i][j] > 10:
            better_viz_array[i][j] = 10
            
plt.figure(figsize=(10,10))
plt.imshow(better_viz_array)
plt.colorbar();

In [26]:
# Ready-to-use visualization cell for bugfixes
# better_viz_array = np.zeros(shape)
# for pixel in pixel_list:
#     better_viz_array[pixel.y][pixel.x] = pixel.vol
# plt.figure(figsize=(15,15))
# plt.imshow(better_viz_array)
# plt.colorbar();

In [27]:
# Visualization, but pretty!
better_viz_array = np.zeros(shape)
for pixel in pixel_list:
    if (reflow_array[pixel.y][pixel.x] >= pond_cutoff) and (pixel.z >= 0):
        better_viz_array[pixel.y][pixel.x] = -0.5
    elif pixel.z < 0:
        better_viz_array[pixel.y][pixel.x] = -1
    else:
        better_viz_array[pixel.y][pixel.x] = pixel.z
plt.figure(figsize=(10,10))
plt.imshow(better_viz_array)
plt.colorbar();

In [28]:
# Beyond this point is EXPERIMENTAL
# Roberts edge detection to determine slopes (does this do what I'd hoped?)
edge_array = roberts(noiseball)

plt.figure(figsize=(10,10))
plt.imshow(edge_array)
plt.colorbar();

In [29]:
# A look at slopes with the ocean covered
better_viz_array = np.zeros(shape)
for pixel in pixel_list:
    if pixel.z < 0:
        better_viz_array[pixel.y][pixel.x] = 0
    else:
        better_viz_array[pixel.y][pixel.x] = edge_array[pixel.y][pixel.x]
for i in range(d):
    for j in range(d):
        if better_viz_array[i][j] > 0.05:
            better_viz_array[i][j] = 0.05
plt.figure(figsize=(10,10))
plt.imshow(better_viz_array)
plt.colorbar();

In [30]:
# I don't remember what this cell was supposed to do - possibly an interaction between volumes and high slopes?
better_viz_array = np.zeros(shape)
for pixel in pixel_list:
    if (reflow_array[pixel.y][pixel.x] >= 10) and (pixel.z > 0):
        better_viz_array[pixel.y][pixel.x] = ((pixel.vol/edge_array[pixel.y][pixel.x])/1000)**0.5
    else:
        better_viz_array[pixel.y][pixel.x] = 0
for i in range(d):
    for j in range(d):
        if better_viz_array[i][j] >= 1:
            better_viz_array[i][j] = 3
        elif pixel_array[i][j].z < 0:
            better_viz_array[i][j] = 1
#         else:
#             better_viz_array[i][j] = 0
for pond in ponds:
    for member in pond.members:
        better_viz_array[member.y][member.x] = 1
plt.figure(figsize=(10,10))
plt.imshow(better_viz_array)
plt.colorbar();

In [31]:
# I believe this cell was an experiment in allowing very large volumes of water to spread to adjacent pixels
better_viz_array = np.zeros(shape)
for pixel in pixel_list:
    if pixel.z < 0:
        better_viz_array[pixel.y][pixel.x] = 0
    else:
        better_viz_array[pixel.y][pixel.x] = reflow_array[pixel.y][pixel.x]
for i in range(d):
    for j in range(d):
        if (better_viz_array[i][j] > 100) and better_viz_array[i]:
             for pixel in pixel_array[i][j].get_adjacents():
                    better_viz_array[pixel.y][pixel.x] = 100
for pond in ponds:
    for member in pond.members:
        better_viz_array[member.y][member.x] = 50
plt.figure(figsize=(10,10))
plt.imshow(better_viz_array)
plt.colorbar();

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()